<a href="https://colab.research.google.com/github/lukemerrick/image-text-embedding/blob/main/initial_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration

In [19]:
!pip install ftfy braceexpand "submitit==1.2.1" "git+https://github.com/facebookresearch/MetaCLIP.git"

  Cloning https://github.com/facebookresearch/MetaCLIP.git to /tmp/pip-req-build-kv4xialp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/MetaCLIP.git /tmp/pip-req-build-kv4xialp
  Resolved https://github.com/facebookresearch/MetaCLIP.git to commit b3695e493ef2d2932809586a179c6609b1234deb
  Preparing metadata (setup.py) ... done
  Created wheel for open_clip_torch: filename=open_clip_torch-0.2-py3-none-any.whl size=1402675 sha256=931dee0a5edde7e52399eef47258f3eed2fcec45d2002ce956e69c3a9fd6d171
  Stored in directory: /tmp/pip-ephem-wheel-cache-498s9fum/wheels/3e/26/0c/d434989e5cf3b4fadb5e07dcb00b7f8c4297c07aad3ea15462
Successfully built open_clip_torch


In [ ]:
# Get code to load the latest MetaCLIP model.
!git clone https://github.com/facebookresearch/MetaCLIP.git ./MetaCLIP
!mv ./MetaCLIP/src ./src
!rm -rf ./MetaCLIP

# Download the latest MetaCLIP model weights.
!wget -q --show-progress https://dl.fbaipublicfiles.com/MMPT/metaclip/h14_v1.2_altogether.pt

Cloning into './MetaCLIP'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 686 (delta 107), reused 129 (delta 70), pack-reused 486 (from 2)
Receiving objects: 100% (686/686), 25.62 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (269/269), done.
h14_v1.2_altogether  59%[==========>         ]   2.18G  29.5MB/s    eta 49s    

In [32]:
from src.open_clip.factory import create_model_and_transforms
from src.open_clip.tokenizer import tokenize

In [ ]:
model, _, preprocess = create_model_and_transforms('ViT-B-32-quickgelu', pretrained='metaclip_400m')

In [1]:
import torch

In [9]:
import pandas as pd

In [10]:
import tqdm

In [21]:
config = {
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 1024,
  "text_config": {
    "hidden_size": 1024,
    "intermediate_size": 4096,
    "model_type": "clip_text_model",
    "num_attention_heads": 16,
    "num_hidden_layers": 24
  },
  "torch_dtype": "float32",
  "transformers_version": "4.34.0",
  "vision_config": {
    "hidden_size": 1280,
    "intermediate_size": 5120,
    "model_type": "clip_vision_model",
    "num_attention_heads": 16,
    "num_hidden_layers": 32,
    "patch_size": 14
  }
}

In [25]:
AutoProcessor?

In [20]:
from PIL import Image
from transformers import AutoProcessor, AutoModel

processor = AutoProcessor.from_pretrained("facebook/metaclip-h14-400m")
model = AutoModel.from_pretrained("facebook/metaclip-b32-400m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:

image = Image.open("docs/CLIP.png")
inputs = processor(text=["a diagram", "a dog", "a cat"], images=image, return_tensors="pt", padding=True)

with torch.no_grad():
  outputs = model(**inputs)
  logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
  text_probs = logits_per_image.softmax(dim=-1)
print("Label probs:", text_probs)